In [ ]:
model_number = 8
csv_file ='model_feature_info.csv'

In [ ]:
import matplotlib.pyplot as plt
# Plot accuracy and loss graphs
def plot_history(history):
    fig, axs = plt.subplots(2, figsize=(10, 10))

    # Plot accuracy
    axs[0].plot(history.history['accuracy'], label='train_accuracy')
    axs[0].plot(history.history['val_accuracy'], label='val_accuracy')
    axs[0].set_title('Accuracy')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Accuracy')
    axs[0].legend()

    # Plot loss
    axs[1].plot(history.history['loss'], label='train_loss')
    axs[1].plot(history.history['val_loss'], label='val_loss')
    axs[1].set_title('Loss')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Loss')
    axs[1].legend()

    plt.tight_layout()
    plt.savefig(f'model number {model_number}.png')
    plt.show()

def write_into_csv(model_params):
    #add extra info into end of model info in csv file
    x = 0
    with open(csv_file, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)

        # Write top-level key-value pairs
        for key, value in model_params.items():
            if x == 0:
              writer.writerow([key, value])
              x+=1
            elif not isinstance(value, dict):  # Check if value is not a dictionary
                writer.writerow([key, value])
            else:  # Handle nested dictionary (data_augmentation)
                for inner_key, inner_value in value.items():
                    writer.writerow([key + '/' + inner_key, inner_value])

In [ ]:
import tensorflow as tf
import time
from tensorflow.keras.datasets import cifar10
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import csv

import numpy as np
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model

In [ ]:
model_number = int(input("enter your last model number: "))
answer_data_augmentation = input('do you want data augmentation:y/n')
answer_one_hot = input('do you want one hot? :y/n')

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
model_number+=1

write_into_csv({' ':' ','model number':model_number})

data_augmentation_params = {
    "vertical_flip": True,
    "rotation_range": 20,
    "zoom_range": 0.2,
    "width_shift_range":0.1,
    "height_shift_range":0.1,
    "horizontal_flip":True
}
model_params = {
    "activation_functions": ["relu","relu","relu","relu","relu","relu","relu","softmax"],
    "batch_size": 32  ,
    "epochs": 200,
    "callbacks": {
        "EarlyStopping": {"monitor": "val_loss", "patience": 15},
        "ModelCheckpoint": {"filepath": "model_checkpoint.keras", "save_best_only": True, "monitor": "val_loss", "verbose": 1},
        "CSVLogger":{"filename":csv_file,"separator":",","append":"True"}
    }
}
# Reduce pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0

# One-hot encode labels (optional, depending on loss function)
number_of_output_class = 10
if answer_one_hot =='y':
  y_train = tf.keras.utils.to_categorical(y_train, number_of_output_class)
  y_test = tf.keras.utils.to_categorical(y_test, number_of_output_class)
else:  
  # flatten the label values
  y_train, y_test = y_train.flatten(), y_test.flatten()



# Build a simple Convolutional Neural Network (CNN)
# model = Sequential([
#     Conv2D(32, (3, 3), activation=model_params['activation_functions'][0], input_shape=(32, 32, 3)),
#     MaxPooling2D((2, 2)),
#     Conv2D(64, (3, 3), activation=model_params['activation_functions'][1]),
#     MaxPooling2D((2, 2)),
#     Conv2D(64, (3, 3),  activation=model_params['activation_functions'][2]),
#     Flatten(),
#     Dense(64,  activation=model_params['activation_functions'][3]),
#     Dense(10,  activation=model_params['activation_functions'][4])
# ])
# model = keras.Sequential(
#     [
#         keras.layers.Input(shape=(32, 32, 3)),
#         keras.layers.Conv2D(32, kernel_size=(3, 3), activation=model_params['activation_functions'][0]),
#         keras.layers.Conv2D(32, kernel_size=(3, 3), activation=model_params['activation_functions'][1]),
#         keras.layers.MaxPooling2D(pool_size=(2, 2)),
#         keras.layers.Conv2D(64, kernel_size=(3, 3), activation=model_params['activation_functions'][2]),
#         keras.layers.Conv2D(64, kernel_size=(3, 3), activation=model_params['activation_functions'][3]),
#         keras.layers.GlobalAveragePooling2D(),
#         keras.layers.Dropout(0.5),
#         keras.layers.Dense(number_of_output_class, activation=model_params['activation_functions'][4]),
#     ]
# )




# number of classes
# K = len(set(y_train))
K = number_of_output_class
 
# Build the model using the functional API
# input layer
i = Input(shape=x_train[0].shape)
x = Conv2D(32, (3, 3), activation=model_params['activation_functions'][0], padding='same')(i)
x = BatchNormalization()(x)
x = Conv2D(32, (3, 3), activation=model_params['activation_functions'][1], padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
 
x = Conv2D(64, (3, 3), activation=model_params['activation_functions'][2], padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation=model_params['activation_functions'][3], padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
 
x = Conv2D(128, (3, 3), activation=model_params['activation_functions'][4], padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation=model_params['activation_functions'][5], padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
 
x = Flatten()(x)
x = Dropout(0.2)(x)
 
# Hidden layer
x = Dense(1024, activation=model_params['activation_functions'][6])(x)
x = Dropout(0.2)(x)
 
# last hidden layer i.e.. output layer
x = Dense(K, activation=model_params['activation_functions'][7])(x)
 
model = Model(i, x)



callbacks = [
    tf.keras.callbacks.EarlyStopping(**model_params['callbacks']['EarlyStopping']),
    tf.keras.callbacks.ModelCheckpoint(**model_params['callbacks']['ModelCheckpoint']),
    # tf.keras.callbacks.ReduceLROnPlateau(**model_params['callbacks']['ReduceLROnPlateau']),
    # tf.keras.callbacks.LearningRateScheduler(**model_params['callbacks']['LearningRateScheduler']),
    tf.keras.callbacks.CSVLogger(**model_params['callbacks']['CSVLogger']),
    tf.keras.callbacks.TerminateOnNaN()
]

# Compile the model
if answer_one_hot =='y':
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
else:
  model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


if answer_data_augmentation =='y':
  model_params["data_augmentation"] = data_augmentation_params
  # datagen = ImageDataGenerator(**model_params['data_augmentation'])
  data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
  rotation_range = 20, zoom_range = 0.2 ,width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True,vertical_flip= True,)
  train_generator = data_generator.flow(x_train, y_train, model_params['batch_size'])
  steps_per_epoch = x_train.shape[0] // model_params['batch_size']

start_time = time.time()

# Train the model
if answer_data_augmentation =='y':
  # history = model.fit(datagen.flow(x_train, y_train, batch_size=model_params['batch_size']), epochs=model_params['epochs'], validation_data=(x_test, y_test) , callbacks=callbacks)
  history= model.fit(train_generator,   ,
              steps_per_epoch=steps_per_epoch, epochs=model_params['epochs'], callbacks=callbacks)
else:
  history = model.fit(x_train, y_train, batch_size=model_params['batch_size'], epochs=model_params['epochs'], validation_data=(x_test, y_test) , callbacks=callbacks)

end_time = time.time()
elapsed_time = end_time - start_time

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
best_val_acc = max(history.history['val_accuracy'])
best_val_loss=min(history.history['val_loss'])
print(f"best Test accuracy: {best_val_acc} ,best test loss :{best_val_loss} ")
write_into_csv({"best test accuracy":best_val_acc,"best test loss":best_val_loss,"elapsed time":elapsed_time})

# Print the model summary
model.summary()
#plot train/val for loss/accuracy
plot_history(history)
write_into_csv(model_params)

#write layer info (flatten or conv2d or maxpooling or ...)
for layer in model.layers:
  layer_info = {
  f"Layer Type":f" {type(layer).__name__}",
  f"Output Shape":f" {layer.output_shape}",
  f"Number of Parameters":f"{layer.count_params()}",
  }
  write_into_csv(layer_info)
model.save('MY_MODEL.keras')